<a href="https://colab.research.google.com/github/ryanvuu/csc-365-lab7/blob/main/csc_365_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libraries

In [1]:
%pip install mysql-connector-python
import getpass
import mysql.connector
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: C:\Users\linki\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### FR1: Rooms and Rates ###

In [2]:
def print_rooms_rates(conn):
    with conn.cursor() as cursor:
        cursor.execute("""
                        WITH PopTable AS (
                            SELECT lab7_rooms.RoomCode,
                                lab7_rooms.RoomName,
                                lab7_rooms.Beds,
                                lab7_rooms.bedType,
                                lab7_rooms.maxOcc,
                                lab7_rooms.basePrice,
                                lab7_rooms.decor,
                                ROUND(SUM(COALESCE(DATEDIFF(lab7_reservations.Checkout, lab7_reservations.CheckIn), 0)) / 180, 2) AS PopScore
                            FROM lab7_rooms
                            LEFT JOIN lab7_reservations
                                ON lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.CheckIn >= DATE_SUB(CURDATE(), INTERVAL 180 DAY)
                            GROUP BY lab7_rooms.RoomCode
                        ), NextAvailable AS (
                            SELECT RoomCode,
                            CASE WHEN
                            -- If a reservation goes past the current date
                            EXISTS (
                                SELECT *
                                FROM lab7_reservations
                                WHERE lab7_reservations.Room = lab7_rooms.RoomCode
                                    AND lab7_reservations.CheckIn <= CURDATE()
                                    AND lab7_reservations.Checkout >= CURDATE()
                            ) THEN (
                                -- Get the earliest checkout date that occurred after the current date
                                SELECT MIN(first_res.Checkout)
                                FROM lab7_reservations AS first_res
                                WHERE first_res.Room = lab7_rooms.RoomCode
                                AND first_res.Checkout > CURDATE()
                                -- See if another guest checked in on the date that the prev. reservaton checked out
                                AND NOT EXISTS (
                                    SELECT *
                                    FROM lab7_reservations as second_res
                                    WHERE first_res.Room = second_res.Room
                                    AND second_res.CheckIn = first_res.Checkout
                                )
                            )
                            ELSE CURDATE()
                            END AS NextAvailableCheckIn
                            FROM lab7_rooms
                        ), RecentReservations AS (
                            SELECT RoomCode,
                            DATEDIFF(
                            (
                                SELECT MAX(Checkout)
                                FROM lab7_reservations
                                WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                    AND lab7_reservations.Checkout <= CURDATE()
                            ),
                            (
                                SELECT MAX(Checkin)
                                FROM lab7_reservations
                                WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.Checkout <= CURDATE() 
                            )
                            ) AS MostRecDuration,
                            (
                            SELECT MAX(Checkout)
                            FROM lab7_reservations
                            WHERE lab7_rooms.RoomCode = lab7_reservations.Room
                                AND lab7_reservations.Checkout <= CURDATE()
                            ) AS MostRecentCheckout
                            FROM lab7_rooms
                        )
                        SELECT *
                        FROM PopTable
                        NATURAL JOIN NextAvailable
                        NATURAL JOIN RecentReservations
                        ORDER BY PopScore DESC;
                        """)
        result = cursor.fetchall()
        print(result)

### FR2: Reservations ###

In [3]:
def make_reservation(conn):
  with conn.cursor() as cursor:
    first_name = input("Enter first name: ")
    last_name = input("Enter last name: ")

    if not first_name.isalpha() or not last_name.isalpha():
      print("Error: Invalid name(s), they should only include letters!")
      return

    room_code = input("Enter room code [OPTIONAL: Any]: ")
    if room_code.capitalize() != "Any":
      cursor.execute("SELECT COUNT(*) FROM rvu03.lab7_rooms WHERE ROOMCODE = %s", [room_code])
      if cursor.fetchone()[0] == 0:
        print("Error: Invalid room code!")
        return

    bed_type = input("Enter bed type [OPTIONAL: Any]: ")
    if bed_type.capitalize() != "Any":
      cursor.execute("SELECT COUNT(*) FROM rvu03.lab7_rooms WHERE bedType = %s", [bed_type])
      if cursor.fetchone()[0] == 0:
        print("Error: Invalid bed type!")
        return
    try:     
      checkin = input("Enter checkin date in the following format YYYY-MM-DD: ")
      checkout = input("Enter checkout date in the following format YYYY-MM-DD: ")

      checkin = datetime.strptime(checkin, "%Y-%m-%d")
      checkout = datetime.strptime(checkout, "%Y-%m-%d")

      if checkin >= checkout:
        print("Invalid checkin and checkout times. Please try again.")
        return
    except ValueError:
        print("Invalid date format, it should be YYYY-MM-DD.")
        return
    
    try:
      num_adults = input("Enter number of adults for reservation: ")
      num_children = input("Enter number of children for reservation: ")

      if not num_adults.isdigit() or not num_children.isdigit():
        print("Error: Invalid number of adults and children must be numerics values.")
      
      num_adults = int(num_adults)
      num_children = int(num_children)

      num_guests = num_adults + num_children
      if num_guests <= 0:
        print("Error: Invalid total number of guests, you need at least one person in the room.")
    except ValueError:
      print("Invalid adults and children numbers!")
      

### FR3: Reservation Cancellation ###

In [4]:
def cancel_reservation(conn):
  with conn.cursor() as cursor:
    print("Delete Reservation")
    print("----------")
    reservation_code = input("Enter Reservation Code: ")
    cursor = conn.cursor()
    cursor.execute("SELECT CODE, Room, CheckIn, Checkout, Rate, LastName, FirstName, Adults, Kids FROM lab7_reservations WHERE CODE = %s", [reservation_code])
    result = cursor.fetchall()
    
    if result:
      print(result)
      confirmation = input("Are you sure you want to cancel this reservation? [Y]es]")

      if confirmation in ("Y", "Yes"):
          cursor.execute("DELETE FROM lab7_reservations WHERE CODE = %s", [reservation_code])
          conn.commit()
          print("You successfully cancelled the reservation. Thank you for considering us!")
      else:
          print("The reservation was not cancelled. Thank you for your staying with us!")
    else:
        print("The reservation code was not found. Please double check your code and try again.")

### FR4: Reservation Information ###

In [ ]:
def reservation_details(conn):
  with conn.cursor() as cursor:
    print("\nDetailed Reservation Information Filters")
    print("----------")
    query = """SELECT Code, Room, RoomName, Beds, BedType, CheckIn, CheckOut, Rate, LastName, FirstName, Adults, Kids 
    FROM lab7_reservations JOIN lab7_rooms ON lab7_rooms.RoomCode = lab7_reservations.Room"""
    filters = []
    inputs = []

    first_name = input("[OPTIONAL] Enter the first name: ")
    if first_name:
      print("First name filter by: ", first_name)
      filters.append('FirstName LIKE %s')
      inputs.append(f"{first_name}")
    
    last_name = input("[OPTIONAL] Enter the last name: ")
    if last_name:
      print("Last name filter by: ", last_name)
      filters.append('LastName LIKE %s')
      inputs.append(f"{last_name}")

    begin_date = input("[OPTIONAL] Enter lower bound of reservation date: ")
    if begin_date:
      print("Lower bound of reservations: ", begin_date)
      filters.append('Checkin >= %s')
      inputs.append(f"{begin_date}")

    end_date = input("[OPTIONAL] Enter upper bound of reservation date: ")
    if end_date:
      print("Upper bound of reservations: ", end_date)
      filters.append('Checkout <= %s' % end_date)
      inputs.append(f"{end_date}")

    room = input("[OPTIONAL] Enter room code: ")    
    if room:
      print("Room code filter by: ", room)
      filters.append('Room LIKE %s')
      inputs.append(f"{room}")

    reserve = input("[OPTIONAL] Enter reservation code: ")    
    if reserve:
      print("Reservation code filter by: ", reserve)
      filters.append('Code LIKE %s')
      inputs.append(f"{reserve}")


    if filters:
      query += " WHERE " + " and ".join(filters)

    cursor.execute(query, inputs)
    result = cursor.fetchall()

    print("Matching Reservations")
    print("----------")
    for row in result:
      print("""Reservation Code: %s, Room Code: %s, Room Name: %s
            Beds: %s, BedType: %s, Check-In: %s, Check-Out: %s
            Rate %s, Last Name: %s, First Name: %s, Adults: %s, Kids: %s \n""" 
            % (row[0], row[1], row[2], 
            row[3], row[4], row[5].strftime('%Y-%m-%d'), row[6].strftime('%Y-%m-%d'),
            row[7], row[8], row[9], row[10], row[11]))

### FR5: Revenue ###

In [6]:
def check_revenue(conn):
  with conn.cursor() as cursor:
    print("Check revenue")
    print("----------")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM lab7_reservations")
    result = cursor.fetchall()
    print(result)
    


# **Main Program**

In [ ]:
def main():
    try:
        db_password = getpass.getpass()
        conn = mysql.connector.connect(user='rvu03', password=db_password,
                               host='mysql.labthreesixfive.com',
                               database='rvu03')
        cursor = conn.cursor()
        while True:
            print("Select a Reservation Option:")
            print("[1] Rooms and Rates")
            print("[2] Reservations")
            print("[3] Reservation Cancellation")
            print("[4] Detailed Reservation Information")
            print("[5] Revenue")
            print("[6] Quit")

            choice = input()
            if choice == "1":
                print_rooms_rates(conn)
            elif choice == "2":
                make_reservation(conn)
            elif choice == "3":
                cancel_reservation(conn)
            elif choice == "4":
                reservation_details(conn)
            elif choice == "5":
                check_revenue(conn)
            elif choice == "6":
                break
    finally:
        cursor.close()
        conn.close()

In [ ]:
if __name__ == "__main__":
    main()